In [ ]:
# build module
!python "../openstorm_radar_py/setup.py" build -j 8

In [ ]:
import sys
import os
import numpy as np
import time
import math
import matplotlib.animation
import IPython
from matplotlib import pyplot as plt
import matplotlib

#%config InlineBackend.figure_formats = ['svg']
#%config InlineBackend.figure_formats = ['jpeg']
%config InlineBackend.figure_formats = ['retina']
#%config InlineBackend.figure_formats = ['png']
plt.rcParams['svg.fonttype'] = 'none'
plt.rcParams["figure.figsize"] = (7, 7)


sys.path.insert(0, '../')
import openstorm_radar_py



In [ ]:
# create RadarDataHolder object
radar_data_holder = openstorm_radar_py.RadarDataHolder()

In [ ]:
# select products to be loaded
reflectivity_product = radar_data_holder.get_product(openstorm_radar_py.VolumeTypes.VOLUME_REFLECTIVITY)
velocity_product = radar_data_holder.get_product(openstorm_radar_py.VolumeTypes.VOLUME_VELOCITY)
# derived products
srv_product = radar_data_holder.get_product(openstorm_radar_py.VolumeTypes.VOLUME_STORM_RELATIVE_VELOCITY)
rotation_product = radar_data_holder.get_product(openstorm_radar_py.VolumeTypes.VOLUME_ROTATION)

In [ ]:
# load file
radar_data_holder.load("../OpenStorm/Content/Data/Demo/KTLX20130531_231434_V06")
#radar_data_holder.load("../files/el-reno/compressed/KTLX20130531_231434_V06.gz")
# wait for it to finish loading because RadarDataHolder is asynchronous and multi-threaded
while(radar_data_holder.get_state() == openstorm_radar_py.RadarDataHolder.DataStateLoading):
	print("loading...", end='\r')
	time.sleep(0.1)
print("loaded      ", end='\n')

In [ ]:
# get now loaded RadarData objects from products
print("is reflectivity loaded?", reflectivity_product.is_loaded(), "   is velocity loaded?", velocity_product.is_loaded(), "   is srv loaded?", srv_product.is_loaded(), "   is rotation loaded?", rotation_product.is_loaded())
reflectivity_data = reflectivity_product.get_radar_data()
velocity_data = velocity_product.get_radar_data()
srv_data = srv_product.get_radar_data()
rotation_data = rotation_product.get_radar_data()
velocity_data.get_stats()

In [ ]:
# a function to correctly plot sweep buffers
def plot_radial_image(buffer):
	theta, rad = np.meshgrid(np.linspace(np.pi * 2.5, np.pi * 0.5, buffer.shape[0]), np.linspace(0, buffer.shape[1], buffer.shape[1]))
	#print(theta.shape, rad.shape)
	fig = plt.figure()
	ax = fig.add_subplot(111, polar='True')
	ax.pcolormesh(theta, rad, np.transpose(buffer), shading='auto')
	plt.show()

In [ ]:
print(reflectivity_data._ptr, velocity_data._ptr, rotation_data._ptr, srv_data._ptr)
print(reflectivity_data.buffer)
#time.sleep(1)
plot_radial_image(np.array(reflectivity_data.buffer)[0,1:-1,:1000])
#time.sleep(0.5)
plot_radial_image(np.array(velocity_data.buffer)[0,1:-1,:1000])
#time.sleep(0.5)
plot_radial_image(np.clip(np.array(srv_data.buffer),-10, 10)[0,1:-1,:1000])
#time.sleep(0.5)
plot_radial_image(np.array(rotation_data.buffer)[0,1:-1,:1000])

In [ ]:
import importlib
import tornado_data
importlib.reload(tornado_data)

# test generating the mask of where tornados are
mask = tornado_data.generate_mask(srv_data)[:,:1000]

fig = plt.figure(figsize=(7,7))
ax = fig.add_subplot()
ax.imshow(mask, interpolation='nearest')
plt.show()
plot_radial_image(mask)

In [ ]:
# make an animation out of the files in a directory and plot tornados as red dots using tornado_mask.py
if 1:
	print("start animation", end='\r')
	fig = plt.figure(figsize=(12,12))
	ax = fig.add_subplot(111, polar='True')

	#animation_data_dir = "../files/el-reno/"
	animation_data_dir = "../files/tornado-2011/"
	animation_data_files = list(map(lambda x: animation_data_dir + x, os.listdir(animation_data_dir)))
	# animation_data_files = animation_data_files[1:]

	def drawframe(frame):
		radius = 1000
		print("frame", frame + 1, "             ", end='\r')
		time.sleep(0.05)
		
		radar_data_holder = openstorm_radar_py.RadarDataHolder()
		
		ref_product = radar_data_holder.get_product(openstorm_radar_py.VolumeTypes.VOLUME_REFLECTIVITY)
		#rot_product = radar_data_holder.get_product(openstorm_radar_py.VolumeTypes.VOLUME_ROTATION)
		srv_product = radar_data_holder.get_product(openstorm_radar_py.VolumeTypes.VOLUME_STORM_RELATIVE_VELOCITY)
		
		radar_data_holder.load(animation_data_files[frame])
		# wait for it to finish loading because RadarDataHolder is asynchronous and multi-threaded
		while(radar_data_holder.get_state() == openstorm_radar_py.RadarDataHolder.DataStateLoading):
			time.sleep(0.1)
		if srv_product.is_loaded() == False:
			return tuple([])
			
		radar_data = ref_product.get_radar_data()
		#radar_data = rot_product.get_radar_data()
		buffer = np.array(radar_data.buffer)[0,1:-1,:radius]
		
		#radar_data = srv_product.get_radar_data()
		#buffer = np.clip(np.array(radar_data.buffer),-20, 20)[0,1:-1,:radius]
		
		print("frame", frame + 1, "loaded       ", end='\r')
		#buffer = np.array(radar_data.buffer)[0,1:-1,:1000]
		theta, rad = np.meshgrid(np.linspace(np.pi * 2.5, np.pi * 0.5, buffer.shape[0]), np.linspace(0, 1, buffer.shape[1]))
		ax.clear()
		ar = ax.pcolormesh(theta, rad, np.transpose(buffer), shading='auto')
		# plot mask overlay
		transparent_cmap = matplotlib.colors.LinearSegmentedColormap.from_list('transparent_cmap',["white","red"],256)
		transparent_cmap._init()
		alphas = np.linspace(0, 0.3, transparent_cmap.N+3)
		transparent_cmap._lut[:,-1] = alphas
		mask = tornado_data.generate_mask(radar_data)[:,:radius]
		ar2 = ax.pcolormesh(theta, rad, np.transpose(mask), shading='auto', cmap=transparent_cmap)
		return tuple([ar, ar2])
	anim = matplotlib.animation.FuncAnimation(fig, drawframe, frames=len(animation_data_files), interval=1000/10, blit=True)
	IPython.display.display(IPython.display.HTML(anim.to_html5_video()))
	plt.close()